In [ ]:
import MySQLdb
import json
conn = MySQLdb.connect(host='202.120.36.29', port=3306, user='groupleader', passwd='onlyleaders', db='mag-new-160205',
                       charset="utf8")
cursor = conn.cursor()

In [14]:
author_names_to_process = list()
for line in open('./AuthorNameIn985.txt'):
    author_names_to_process.append(line.replace('\n', ''))

In [ ]:
with open('paper_affliliations.txt','w') as f:
    for author_name in author_names_to_process:
        quest_paper_by_author_name = 'SELECT PaperID,AffiliationID,A.AuthorID FROM PaperAuthorAffiliations AS P INNER JOIN ' \
                                     '(SELECT AuthorID FROM Authors WHERE AuthorName ="%s") AS A ' \
                                     'ON P.AuthorID = A.AuthorID'
        cursor.execute(quest_paper_by_author_name % author_name)
        paper_affiliations = cursor.fetchall() 
        if 200 < len(paper_affiliations) < 300:
            f.write(author_name+':')
            #print 'yes
            for item in paper_affiliations:
                paper_id = item[0]
                quest_author_by_paper = 'SELECT AuthorName FROM Authors INNER JOIN ' \
                            '(SELECT AuthorID FROM PaperAuthorAffiliations WHERE PaperID = "%s") AS TB ' \
                            'ON Authors.AuthorID = TB.AuthorID'
                cursor.execute(quest_author_by_paper % paper_id)
                coauthors = cursor.fetchall()
                if len(coauthors) < 20:
                    quest_info_by_paper = 'SELECT NormalizedPaperTitle, ConferenceSeriesIDMappedToVenueName, ' \
                          'JournalIDMappedToVenueName, PaperPublishYear FROM Papers WHERE PaperID = "%s"'
                    cursor.execute(quest_info_by_paper % paper_id)
                    rs = cursor.fetchall()
                    f.write(str(item) + '\s' +str(coauthors) + '\s'+ str(rs) +'\t')
            f.write('\n')  

In [ ]:
with open('name_dis.json','w') as f:
    for author_name in author_names_to_process:
        if len(author_name.split()) >= 2:
            quest_paper_by_author_name = 'SELECT PaperID,AffiliationID,A.AuthorID FROM PaperAuthorAffiliations AS P INNER JOIN ' \
                                     '(SELECT AuthorID FROM Authors WHERE AuthorName ="%s") AS A ' \
                                     'ON P.AuthorID = A.AuthorID'
            cursor.execute(quest_paper_by_author_name % author_name)
            paper_affiliations = cursor.fetchall()
            if 200 < len(paper_affiliations) < 300:
                a = []
                #print 'yes
                for paper_affiliation in paper_affiliations:
                    paper_id = paper_affiliation[0]
                    quest_author_by_paper = 'SELECT AuthorName FROM Authors INNER JOIN ' \
                                '(SELECT AuthorID FROM PaperAuthorAffiliations WHERE PaperID = "%s") AS TB ' \
                                'ON Authors.AuthorID = TB.AuthorID'
                    cursor.execute(quest_author_by_paper % paper_id)
                    coauthors = cursor.fetchall()
                    if len(coauthors) < 20:
                        quest_info_by_paper = 'SELECT NormalizedPaperTitle, ConferenceSeriesIDMappedToVenueName, ' \
                              'JournalIDMappedToVenueName, PaperPublishYear FROM Papers WHERE PaperID = "%s"'
                        cursor.execute(quest_info_by_paper % paper_id)
                        rs = cursor.fetchall()
                        a.append((paper_affiliation,{'coauthor':coauthors,'title_venue_year':rs})
                dict = {'author_name':author_name,'paper_affiliations':a}
                json.dump(dict,f)
                f.write('\n')